# Original FRCNN

Files uploaded from this model are:
```
JunKai_final_evaluation_roboflow_22182_submission.json
JunKai_final_evaluation_roboflow_28182_submission.json
```
 - The numbers represent the training steps done
 - The notebook has went through changes while training


Tutorial adapted from Roboflow - frcnn code

[Notebook](https://colab.research.google.com/drive/1U3fkRu6-hwjk7wWIpg-iylL2u5T9t7rr#scrollTo=uQCnYPVDrsgx)

[Article](https://towardsdatascience.com/training-a-tensorflow-faster-r-cnn-object-detection-model-on-your-own-dataset-b3b175708d6d)


In [ ]:
!nvidia-smi

Fri Jun 19 19:00:38 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P0    38W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install tensorflow_gpu==1.15

## Configs and Hyperparameters

Support a variety of models, you can find more pretrained model from [Tensorflow detection model zoo: COCO-trained models](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md#coco-trained-models), as well as their pipline config files in [object_detection/samples/configs/](https://github.com/tensorflow/models/tree/master/research/object_detection/samples/configs).

In [ ]:
# If you forked the repo, you can replace the link.
repo_url = 'https://github.com/roboflow-ai/tensorflow-object-detection-faster-rcnn'

# Number of training steps - 1000 will train very quickly, but more steps will increase accuracy.
num_steps = 200000  # 200000 to improve

# Number of evaluation steps.
num_eval_steps = 50

MODELS_CONFIG = {
    'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_coco_2018_03_29',
        'pipeline_file': 'ssd_mobilenet_v2_coco.config',
        'batch_size': 12
    },
    'faster_rcnn_inception_v2': {
        'model_name': 'faster_rcnn_inception_v2_coco_2018_01_28',
        'pipeline_file': 'faster_rcnn_inception_v2_pets.config',
        'batch_size': 12
    },
    'rfcn_resnet101': {
        'model_name': 'rfcn_resnet101_coco_2018_01_28',
        'pipeline_file': 'rfcn_resnet101_pets.config',
        'batch_size': 8
    }
}

# Pick the model you want to use
# Select a model in `MODELS_CONFIG`.
selected_model = 'faster_rcnn_inception_v2'

# Name of the object detection model to use.
MODEL = MODELS_CONFIG[selected_model]['model_name']

# Name of the pipline file in tensorflow object detection API.
pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']

# Training batch size fits in Colabe's Tesla K80 GPU memory for selected model.
batch_size = MODELS_CONFIG[selected_model]['batch_size']

## Clone the `tensorflow-object-detection` repository or your fork.

In [ ]:
import os

%cd /content

repo_dir_path = os.path.abspath(os.path.join('.', os.path.basename(repo_url)))

!git clone {repo_url}
%cd {repo_dir_path}
!git pull

/content
fatal: destination path 'tensorflow-object-detection-faster-rcnn' already exists and is not an empty directory.
/content/tensorflow-object-detection-faster-rcnn
Already up to date.


## Install required packages

In [ ]:
%cd /content
!git clone --quiet https://github.com/tensorflow/models.git

!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

!pip install -q Cython contextlib2 pillow lxml matplotlib

!pip install -q pycocotools

%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.

import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'

!pip install tf_slim

!python object_detection/builders/model_builder_test.py

/content
fatal: destination path 'models' already exists and is not an empty directory.
/content/models/research
object_detection/protos/input_reader.proto: warning: Import object_detection/protos/image_resizer.proto but not used.


##Prepare tfrecord files

In [ ]:
# NOTE: Update these TFRecord names from "cells" and "cells_label_map" to your files!
test_record_fname = '/content/drive/My Drive/Colab Notebooks/TIL/CV/input/tfrecord/coco_val.record-00000-of-00001'
train_record_fname = '/content/drive/My Drive/Colab Notebooks/TIL/CV/input/tfrecord/coco_train.record-00000-of-00001'
label_map_pbtxt_fname = '/content/drive/My Drive/Colab Notebooks/TIL/CV/input/tfrecord/tfrecord_label_map.pbtxt'

## Download base model

In [ ]:
%cd /content/models/research

import os
import shutil
import glob
import urllib.request
import tarfile
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = '/content/models/research/pretrained_model'

if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

/content/models/research


In [ ]:
!echo {DEST_DIR}
!ls -alh {DEST_DIR}

/content/models/research/pretrained_model
total 111M
drwxr-xr-x  3 345018 5000 4.0K Feb  1  2018 .
drwxr-xr-x 64 root   root 4.0K Jun 19 17:45 ..
-rw-r--r--  1 345018 5000   77 Feb  1  2018 checkpoint
-rw-r--r--  1 345018 5000  55M Feb  1  2018 frozen_inference_graph.pb
-rw-r--r--  1 345018 5000  51M Feb  1  2018 model.ckpt.data-00000-of-00001
-rw-r--r--  1 345018 5000  16K Feb  1  2018 model.ckpt.index
-rw-r--r--  1 345018 5000 5.5M Feb  1  2018 model.ckpt.meta
-rw-r--r--  1 345018 5000 3.2K Feb  1  2018 pipeline.config
drwxr-xr-x  3 345018 5000 4.0K Feb  1  2018 saved_model


In [ ]:
fine_tune_checkpoint = os.path.join(DEST_DIR, "model.ckpt")
fine_tune_checkpoint

'/content/models/research/pretrained_model/model.ckpt'

## Configuring a Training Pipeline

In [ ]:
model_dir = '/content/drive/My Drive/Colab Notebooks/TIL/CV/junkai/Roboflow_tf_save_folder/'
model_dir = os.path.join('/',*model_dir.split('/'))


print(model_dir)  # model_dir needs to be surrounded by "" when sent to the command line

# Optionally remove content in output model directory to fresh start.
# !rm -rf "{model_dir}"
os.makedirs(model_dir, exist_ok=True)

/content/drive/My Drive/Colab Notebooks/TIL/CV/junkai/Roboflow_tf_save_folder/


In [ ]:
import re

query = model_dir + "model.ckpt-"
latest_ckpt = !ls "{query}"*

ckpt_num = []
for i in latest_ckpt:
  ckpt_num.append(int(re.findall(r'\d{4,6}', i)[0]))
latest_ckpt = str(max(ckpt_num))

# load checkpoints in. If not resuming from a model, comment this out
fine_tune_checkpoint = os.path.join(model_dir, "model.ckpt-" + latest_ckpt)

fine_tune_checkpoint

'/content/drive/My Drive/Colab Notebooks/TIL/CV/junkai/Roboflow_tf_save_folder/model.ckpt-20182'

In [ ]:
import os
pipeline_fname = os.path.join('/content/models/research/object_detection/samples/configs/', pipeline_file)

assert os.path.isfile(pipeline_fname), '`{}` not exist'.format(pipeline_fname)

In [ ]:
def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

In [ ]:
get_num_classes(label_map_pbtxt_fname)

5

In [ ]:
# Re-writing the pipeline_fname file to know which directory contains what
with open("/content/models/research/object_detection/model_main.py") as f:
    s = f.read()
with open("/content/models/research/object_detection/model_main.py", 'w') as f:
    s = re.sub(re.escape("config = tf.estimator.RunConfig(model_dir=FLAGS.model_dir)"), 
               "config = tf.estimator.RunConfig(model_dir=FLAGS.model_dir, save_checkpoints_steps=2000)", s)
        
    f.write(s)

!cat /content/models/research/object_detection/model_main.py

# Copyright 2017 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""Binary to run train and evaluation on object detection model."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from absl import flags

import tensorflow.compat.v1 as tf

from object_detection import model_hparams
from object_detection import m

In [ ]:
num_classes = get_num_classes(label_map_pbtxt_fname)  # Can actually just sub with no. of classes

# Re-writing the pipeline_fname file to know which directory contains what
with open(pipeline_fname) as f:
    s = f.read()
with open(pipeline_fname, 'w') as f:

    # fixed_shape_resizer - changed as aspect_ratio_resizer causes image shape errors
    # Refer to: https://github.com/tensorflow/tensorflow/issues/34544
    s = re.sub('''keep_aspect_ratio_resizer {
        min_dimension: 600
        max_dimension: 1024
      }''', 
      '''fixed_shape_resizer {
        height: 600
        width: 800
      }''', s)
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)

    # eval_num_examples
    s = re.sub('num_examples: \d*',
               'num_examples: {}'.format(1), s)

    # # # Set scales - deleted 0.25 (scale*width - 0.25 is for very small objects)
    # s = re.sub('scales: \\[.*\\]',
    #            'scales: [0.25, 0.5, 1.0, 2.0]', s)
    
    # # Set aspect_ratios - deleted 2.0 (width>height)
    # s = re.sub('aspect_ratios: \\[.*\\]',
    #            'aspect_ratios: [0.5, 1.0, 2.0]', s)
    
    # Number of proposals (previously 300) in second generator
    s = re.sub('max_total_detections: \d*',
               'max_total_detections: {}'.format(150), s)
    
    # Learn rate  
    s = re.sub('step: \d*\n            learning_rate: .00002',
               'step: {}\n            learning_rate: {}'.format(20000, .00004), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(train.record)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(val.record)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # # num_readers
    # s = re.sub(
    #     'train_input_reader: {(\n  num_readers: 100)?', r'train_input_reader: {\n  num_readers: 100', s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)
    
    # Set training steps, num_steps - run indefinitely as one time is only 2000 steps
    s = re.sub('num_steps: [0-9]+',
               '', s)
              #  'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    f.write(s)

!cat {pipeline_fname}

# Faster R-CNN with Inception v2, configured for Oxford-IIIT Pets Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  faster_rcnn {
    num_classes: 5
    image_resizer {
      fixed_shape_resizer {
        height: 600
        width: 800
      }
    }
    feature_extractor {
      type: 'faster_rcnn_inception_v2'
      first_stage_features_stride: 16
    }
    first_stage_anchor_generator {
      grid_anchor_generator {
        scales: [0.25, 0.5, 1.0, 2.0]
        aspect_ratios: [0.5, 1.0, 2.0]
        height_stride: 16
        width_stride: 16
      }
    }
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {
        l2_regularizer {
          weight: 0.0
        }
      }
      initializer {
        truncated_normal_initiali

## Run Tensorboard(Optional)

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

--2020-06-19 19:03:12--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 3.223.98.104, 34.193.189.199, 3.224.101.150, ...
Connecting to bin.equinox.io (bin.equinox.io)|3.223.98.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.2’

ngrok-stable-linux- 100%[===================>]  13.13M  57.6MB/s    in 0.2s    

2020-06-19 19:03:12 (57.6 MB/s) - ‘ngrok-stable-linux-amd64.zip.2’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
LOG_DIR = model_dir

get_ipython().system_raw(
    'tensorboard --logdir "{}" --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [ ]:
get_ipython().system_raw('./ngrok http 6006 &')

### Get Tensorboard link

In [ ]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://d335ac8d56af.ngrok.io


## Train the model

In [ ]:
!python -m pip uninstall numpy
!python -m pip install numpy==1.17

Uninstalling numpy-1.17.0:
  Would remove:
    /usr/local/bin/f2py
    /usr/local/bin/f2py3
    /usr/local/bin/f2py3.6
    /usr/local/lib/python3.6/dist-packages/numpy-1.17.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/numpy/*
Proceed (y/n)? ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/pip/_internal/cli/base_command.py", line 153, in _main
    status = self.run(options, args)
  File "/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/uninstall.py", line 79, in run
    auto_confirm=options.yes, verbose=self.verbosity > 0,
  File "/usr/local/lib/python3.6/dist-packages/pip/_internal/req/req_install.py", line 755, in uninstall
    uninstalled_pathset.remove(auto_confirm, verbose)
  File "/usr/local/lib/python3.6/dist-packages/pip/_internal/req/req_uninstall.py", line 388, in remove
    if auto_confirm or self._allowed_to_proceed(verbose):
  File "/usr/local/lib/python3.6/dist-packages/pip/_intern

KeyboardInterrupt: ignored

In [ ]:
# Suppress deprecation warnings: https://stackoverflow.com/questions/40426502/is-there-a-way-to-suppress-the-messages-tensorflow-prints/40426709
# import tensorflow as tf
# tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

[autoreload of numpy.core.multiarray failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
RuntimeError: empty_like method already has a docstring
]
[autoreload of numpy.core.overrides failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
RuntimeError: implement_array_function method already has a docstring
]
[autoreload of numpy.lib failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
NameError: name 'type_check' is not defined
]
[autoreload of numpy.lib.scimath failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/extensions/autoreload.py", l

In [ ]:
import traceback

try:
  !python /content/models/research/object_detection/model_main.py \
      --pipeline_config_path={pipeline_fname} \
      --model_dir="{model_dir}" \
      --alsologtostderr \
      --num_train_steps=100000 \
      --num_eval_steps={num_eval_steps}
except Exception as e:
  traceback.print_exc()

W0619 18:02:43.805433 140682311186304 model_lib.py:717] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting train_steps: 100000
I0619 18:02:43.805664 140682311186304 config_util.py:523] Maybe overwriting train_steps: 100000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0619 18:02:43.805762 140682311186304 config_util.py:523] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1
I0619 18:02:43.805839 140682311186304 config_util.py:523] Maybe overwriting sample_1_of_n_eval_examples: 1
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I0619 18:02:43.805917 140682311186304 config_util.py:523] Maybe overwriting eval_num_epochs: 1
INFO:tensorflow:Maybe overwriting load_pretrained: True
I0619 18:02:43.805987 140682311186304 config_util.py:523] Maybe overwriting load_pretrained: True
INFO:tensorflow:Ignoring config override key: load_pretrained
I0619 18:02:43.806055 140682311186304 config_util.

KeyboardInterrupt: ignored

In [ ]:
query = model_dir + "model.ckpt-"
latest_ckpt = !ls "{query}"*

ckpt_num = []
for i in latest_ckpt:
  ckpt_num.append(int(re.findall(r'\d{4,6}', i)[0]))
latest_ckpt = str(max(ckpt_num))

# load checkpoints in. If not resuming from a model, comment this out
fine_tune_checkpoint = os.path.join(model_dir, "model.ckpt-" + latest_ckpt)

print(fine_tune_checkpoint)

# Re-writing the pipeline_fname file to know which directory contains what
with open(pipeline_fname) as f:
    s = f.read()
with open(pipeline_fname, 'w') as f:
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    f.write(s)

/content/drive/My Drive/Colab Notebooks/TIL/CV/junkai/Roboflow_tf_save_folder/model.ckpt-26182


In [ ]:
import traceback

try:
  !python /content/models/research/object_detection/model_main.py \
      --pipeline_config_path={pipeline_fname} \
      --model_dir="{model_dir}" \
      --alsologtostderr \
      --num_train_steps=100000 \
      --num_eval_steps={num_eval_steps}
except Exception as e:
  traceback.print_exc()

W0619 22:06:30.942545 140656273385344 model_lib.py:717] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting train_steps: 100000
I0619 22:06:30.942827 140656273385344 config_util.py:523] Maybe overwriting train_steps: 100000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0619 22:06:30.942948 140656273385344 config_util.py:523] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1
I0619 22:06:30.943048 140656273385344 config_util.py:523] Maybe overwriting sample_1_of_n_eval_examples: 1
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I0619 22:06:30.943160 140656273385344 config_util.py:523] Maybe overwriting eval_num_epochs: 1
INFO:tensorflow:Maybe overwriting load_pretrained: True
I0619 22:06:30.943253 140656273385344 config_util.py:523] Maybe overwriting load_pretrained: True
INFO:tensorflow:Ignoring config override key: load_pretrained
I0619 22:06:30.943359 140656273385344 config_util.

In [ ]:
query = model_dir + "model.ckpt-"
latest_ckpt = !ls "{query}"*

ckpt_num = []
for i in latest_ckpt:
  ckpt_num.append(int(re.findall(r'\d{4,6}', i)[0]))
latest_ckpt = str(max(ckpt_num))

# load checkpoints in. If not resuming from a model, comment this out
fine_tune_checkpoint = os.path.join(model_dir, "model.ckpt-" + latest_ckpt)
print(fine_tune_checkpoint)

# Re-writing the pipeline_fname file to know which directory contains what
with open(pipeline_fname) as f:
    s = f.read()
with open(pipeline_fname, 'w') as f:
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    f.write(s)

/content/drive/My Drive/Colab Notebooks/TIL/CV/junkai/Roboflow_tf_save_folder/model.ckpt-28182


In [ ]:
import traceback

try:
  !python /content/models/research/object_detection/model_main.py \
      --pipeline_config_path={pipeline_fname} \
      --model_dir="{model_dir}" \
      --alsologtostderr \
      --num_train_steps=100000 \
      --num_eval_steps={num_eval_steps}
except Exception as e:
  traceback.print_exc()

W0619 20:35:15.822454 140705607268224 model_lib.py:717] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting train_steps: 100000
I0619 20:35:15.822721 140705607268224 config_util.py:523] Maybe overwriting train_steps: 100000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0619 20:35:15.822839 140705607268224 config_util.py:523] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1
I0619 20:35:15.822933 140705607268224 config_util.py:523] Maybe overwriting sample_1_of_n_eval_examples: 1
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I0619 20:35:15.823017 140705607268224 config_util.py:523] Maybe overwriting eval_num_epochs: 1
INFO:tensorflow:Maybe overwriting load_pretrained: True
I0619 20:35:15.823093 140705607268224 config_util.py:523] Maybe overwriting load_pretrained: True
INFO:tensorflow:Ignoring config override key: load_pretrained
I0619 20:35:15.823172 140705607268224 config_util.

In [ ]:
# /content/models/research/training
!ls "{model_dir}"

checkpoint
events.out.tfevents.1592502322.d0689bcbed25
events.out.tfevents.1592504735.2fc38715b6c7
events.out.tfevents.1592537750.a96c07c4ba08
events.out.tfevents.1592548856.6ea3e0d90724
events.out.tfevents.1592554353.5a883e8ab544
events.out.tfevents.1592557332.2bd3a015e27d
events.out.tfevents.1592561699.60e7df0381c7
events.out.tfevents.1592565360.60e7df0381c7
events.out.tfevents.1592568171.60e7df0381c7
events.out.tfevents.1592570694.60e7df0381c7
events.out.tfevents.1592574084.b72f72a14ca8
events.out.tfevents.1592576938.b72f72a14ca8
events.out.tfevents.1592579247.b72f72a14ca8
events.out.tfevents.1592582798.b72f72a14ca8
events.out.tfevents.1592585655.b72f72a14ca8
graph.pbtxt
model.ckpt-20000.data-00000-of-00001
model.ckpt-20000.index
model.ckpt-20000.meta
model.ckpt-22000.data-00000-of-00001
model.ckpt-22000.index
model.ckpt-22000.meta
model.ckpt-24000.data-00000-of-00001
model.ckpt-24000.index
model.ckpt-24000.meta
model.ckpt-26000.data-00000-of-00001
model.ckpt-26000.index
model.ckpt-

## Exporting a Trained Inference Graph
Once your training job is complete, you need to extract the newly trained inference graph, which will be later used to perform the object detection. This can be done as follows:

In [ ]:
import re
import numpy as np

output_directory = './fine_tuned_model'

lst = os.listdir(model_dir)
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')

last_model_path = os.path.join(model_dir, last_model)
print(last_model_path)
!python /content/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path={pipeline_fname} \
    --output_directory="{output_directory}" \
    --trained_checkpoint_prefix="{last_model_path}"

/content/drive/My Drive/Colab Notebooks/TIL/CV/junkai/Roboflow_tf_save_folder/model.ckpt-28182
Instructions for updating:
Please use `layer.__call__` method instead.
W0619 22:52:43.780848 139685731977088 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tf_slim/layers/layers.py:2802: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:Scale of 0 disables regularizer.
I0619 22:52:45.384879 139685731977088 regularizers.py:99] Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
I0619 22:52:45.401483 139685731977088 regularizers.py:99] Scale of 0 disables regularizer.
INFO:tensorflow:depth of additional conv before box predictor: 0
I0619 22:52:45.401970 139685731977088 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
Instructions for updating:
Use tf.where in 2.0

In [ ]:
!ls {output_directory}

checkpoint			model.ckpt.index  saved_model
frozen_inference_graph.pb	model.ckpt.meta
model.ckpt.data-00000-of-00001	pipeline.config


## Download the model `.pb` file

In [ ]:
import os

pb_fname = os.path.join(os.path.abspath(output_directory), "frozen_inference_graph.pb")
assert os.path.isfile(pb_fname), '`{}` not exist'.format(pb_fname)

In [ ]:
!ls -alh {pb_fname}

-rw-r--r-- 1 root root 50M Jun 19 22:53 /content/models/research/fine_tuned_model/frozen_inference_graph.pb


In [ ]:
query = model_dir + "model.ckpt-"
latest_ckpt = !ls "{query}"*|sort -rn|head -1
latest_ckpt = re.search(r'\d{4,6}', latest_ckpt[0])[0]

save_model_dir = "/content/drive/My Drive/Colab Notebooks/TIL/CV/junkai/roboflow_faster_rcnn_inception_v2_" + latest_ckpt + ".pb"
save_model_dir

'/content/drive/My Drive/Colab Notebooks/TIL/CV/junkai/roboflow_faster_rcnn_inception_v2_28182.pb'

In [ ]:
!cp /content/models/research/fine_tuned_model/frozen_inference_graph.pb "{save_model_dir}"

##Exploratory analysis

In [ ]:
with open("/content/drive/My Drive/Colab Notebooks/TIL/CV/working/submission-model-7x7-14x14-3aspect-modyoloposneg-wd0.0005.json") as f:
  valannos = json.load(f)
with open(submit_annotations) as f:
  subannos = json.load(f)

In [ ]:
valannos[0]

{'bbox': [571.5, 194.6, 815.2, 821.1],
 'category_id': 4,
 'image_id': 1,
 'score': 0.6992745399475098}

In [ ]:
subannos[0]

{'bbox': [962.0, 693.2, 764.6, 970.4],
 'category_id': 4,
 'image_id': 1,
 'score': 0.8713104724884033}

In [ ]:
a=0
for i in valannos:
  if i['image_id'] == 1:  # 2084, 1472, 3030
    a+=1
    print (i)
print(a)

{'image_id': 1, 'category_id': 4, 'bbox': [571.5, 194.6, 815.2, 821.1], 'score': 0.6992745399475098}
{'image_id': 1, 'category_id': 4, 'bbox': [731.3, 128.9, 530.9, 857.4], 'score': 0.11952656507492065}
{'image_id': 1, 'category_id': 4, 'bbox': [668.4, 226.5, 613.6, 971.3], 'score': 0.03665538132190704}
{'image_id': 1, 'category_id': 3, 'bbox': [879.9, 223.0, 901.2, 669.5], 'score': 0.014186333864927292}
{'image_id': 1, 'category_id': 4, 'bbox': [582.3, 108.9, 709.3, 917.1], 'score': 0.01228678971529007}
{'image_id': 1, 'category_id': 4, 'bbox': [632.7, 57.0, 704.6, 505.9], 'score': 0.011506722308695316}
{'image_id': 1, 'category_id': 3, 'bbox': [1292.6, 144.2, 382.8, 665.6], 'score': 0.004970135632902384}
{'image_id': 1, 'category_id': 4, 'bbox': [891.0, 32.2, 655.1, 426.8], 'score': 0.00437589455395937}
{'image_id': 1, 'category_id': 5, 'bbox': [759.9, 550.0, 495.9, 805.6], 'score': 0.0040878006257116795}
{'image_id': 1, 'category_id': 2, 'bbox': [769.7, 842.0, 491.4, 654.4], 'score'

In [ ]:
a=0
for i in subannos:
  if i['image_id'] == 1:  # 2037, 4601, 3921
    a+=1
    print (i)
print(a)

{'image_id': 1, 'category_id': 4, 'bbox': [962.0, 693.2, 764.6, 970.4], 'score': 0.8713104724884033}
{'image_id': 1, 'category_id': 4, 'bbox': [937.1, 735.4, 888.6, 1406.4], 'score': 0.21724474430084229}
{'image_id': 1, 'category_id': 4, 'bbox': [939.6, 815.1, 561.5, 789.0], 'score': 0.16690513491630554}
{'image_id': 1, 'category_id': 3, 'bbox': [997.4, 412.1, 957.7, 484.7], 'score': 0.16587480902671814}
{'image_id': 1, 'category_id': 2, 'bbox': [883.2, 980.4, 603.7, 750.5], 'score': 0.12081144005060196}
{'image_id': 1, 'category_id': 5, 'bbox': [916.4, 791.5, 859.6, 694.5], 'score': 0.10853636264801025}
{'image_id': 1, 'category_id': 4, 'bbox': [773.3, 786.2, 918.4, 1108.2], 'score': 0.0994483083486557}
{'image_id': 1, 'category_id': 1, 'bbox': [842.4, 485.7, 623.4, 568.5], 'score': 0.08482954651117325}
{'image_id': 1, 'category_id': 2, 'bbox': [847.4, 1083.8, 905.5, 1720.4], 'score': 0.08160553127527237}
{'image_id': 1, 'category_id': 3, 'bbox': [940.7, 632.7, 967.2, 636.6], 'score':

In [ ]:
# To fix multiple, we introduce non-maximum suppression, or NMS for short
def nms(detections, iou_thresh=0.):
  dets_by_class = {}
  final_result = []
  for det in detections:
    cls = det[1]
    if cls not in dets_by_class:
      dets_by_class[cls] = []
    dets_by_class[cls].append( det )
  for _, dets in dets_by_class.items():
    candidates = list(dets)
    candidates.sort( key=lambda x:x[0], reverse=True )
    while len(candidates) > 0:
      candidate = candidates.pop(0)
      _,_,cx,cy,cw,ch = candidate
      copy = list(candidates)
      for other in candidates:
        # Compute the IoU. If it exceeds thresh, we remove it
        _,_,ox,oy,ow,oh = other
        if iou( (cx,cy,cw,ch), (ox,oy,ow,oh) ) > iou_thresh:
          copy.remove(other)
      candidates = list(copy)
      final_result.append(candidate)
  return final_result

# Computes the intersection-over-union (IoU) of two bounding boxes
def iou(bb1, bb2):
  x1,y1,w1,h1 = bb1
  xmin1 = x1 - w1/2
  xmax1 = x1 + w1/2
  ymin1 = y1 - h1/2
  ymax1 = y1 + h1/2

  x2,y2,w2,h2 = bb2
  xmin2 = x2 - w2/2
  xmax2 = x2 + w2/2
  ymin2 = y2 - h2/2
  ymax2 = y2 + h2/2

  area1 = w1*h1
  area2 = w2*h2

  # Compute the boundary of the intersection
  xmin_int = max( xmin1, xmin2 )
  xmax_int = min( xmax1, xmax2 )
  ymin_int = max( ymin1, ymin2 )
  ymax_int = min( ymax1, ymax2 )
  intersection = max(xmax_int - xmin_int, 0) * max( ymax_int - ymin_int, 0 )

  # Remove the double counted region
  union = area1+area2-intersection

  return intersection / union


In [ ]:
# Display some of the images
from collections import OrderedDict
from PIL import ImageEnhance, ImageFont, ImageDraw

# Double check if this is still valid
cat_list = ['tops', 'trousers', 'outerwear', 'dresses', 'skirts']

with open(submit_annotations, 'r') as f:
  results = json.load(f)

sorted_results = OrderedDict()
for i in results:
  img_id = i['image_id']

  if img_id in sorted_results:
    sorted_results[img_id].append(i)
  else:
    sorted_results[img_id] = [i]
  
# sorted_results = OrderedDict({img_id_1: {}, img_id_2: {}, ...})

# Run this to visualize
rank_colors = ['cyan', 'magenta', 'DarkOrange', 'DimGray', 'DarkTurquoise']
det_threshold=0.
top_dets=3

start=0
end=20
for k in range(start,end):
  image_id = list(sorted_results.keys())[k]
  image = PIL.Image.open(os.path.join(eval_imgs_folder, str(image_id)+'.jpg'))

  detection = []
  for image_dict in sorted_results[image_id]:
    score = image_dict['score']
    cat_id = image_dict['category_id']
    x,y,w,h = image_dict['bbox']  # [x,y,width,height]

    detection.append((score, cat_id, x, y, w, h))

  # nms takes in and returns [(score, cat, bbox,bbox1,bbox1,bbox1), (...)]
  preds = nms(detection, iou_thresh=0.01)  # Originally 0.5

  for num, pred in enumerate(preds):
    score = pred[0]
    cat_id = pred[1]
    x,y,w,h = pred[2:6]

    x0 = int(x - w/2)
    x1 = int(x + w/2)
    y0 = int(y - h/2)
    y1 = int(y + h/2)
    text = cat_list[cat_id-1]
    score = str(round(score, 5))

    draw = ImageDraw.Draw(image)
    draw.rectangle([x0, y0, x1, y1], outline = rank_colors[cat_id-1])
    draw.text([x1, y1], text, fill = rank_colors[cat_id-1])
    draw.text([x1, y1+15], score, fill = rank_colors[cat_id-1])
  display(image, width=10, height=10)

In [ ]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(
    label_map, max_num_classes=num_classes, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

def run_inference_for_single_image(image, graph):
    with graph.as_default():
        with tf.Session() as sess:
            # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {
                output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in [
                'num_detections', 'detection_boxes', 'detection_scores',
                'detection_classes', 'detection_masks'
            ]:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                        tensor_name)
            if 'detection_masks' in tensor_dict:
                # The following processing is only for single image
                detection_boxes = tf.squeeze(
                    tensor_dict['detection_boxes'], [0])
                detection_masks = tf.squeeze(
                    tensor_dict['detection_masks'], [0])
                # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                real_num_detection = tf.cast(
                    tensor_dict['num_detections'][0], tf.int32)
                detection_boxes = tf.slice(detection_boxes, [0, 0], [
                                           real_num_detection, -1])
                detection_masks = tf.slice(detection_masks, [0, 0, 0], [
                                           real_num_detection, -1, -1])
                detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                    detection_masks, detection_boxes, image.shape[0], image.shape[1])
                detection_masks_reframed = tf.cast(
                    tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                # Follow the convention by adding back the batch dimension
                tensor_dict['detection_masks'] = tf.expand_dims(
                    detection_masks_reframed, 0)
            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

            # Run inference
            output_dict = sess.run(tensor_dict,
                                   feed_dict={image_tensor: np.expand_dims(image, 0)})

            # all outputs are float32 numpy arrays, so convert types as appropriate
            output_dict['num_detections'] = int(
                output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict[
                'detection_classes'][0].astype(np.uint8)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
            if 'detection_masks' in output_dict:
                output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict

# Custom sequence to run the evaluation annotation file

class TILSequence(Sequence):
  def __init__(self, img_folder, json_annotation_file, batch_size, augment_fn, testmode=True):

    self._prepare_data(img_folder, json_annotation_file)
    self.batch_size = batch_size
    self.augment_fn = augment_fn
    self.testmode = testmode

  def _prepare_data(self, img_folder, json_annotation_file):
    imgs_dict = {im.split('.')[0]:im for im in os.listdir(img_folder) if im.endswith('.jpg')}
    data_dict = {}
    with open(json_annotation_file, 'r') as f:
      annotations_dict = json.load(f)
    annotations_list = annotations_dict['images']
    for annotation in annotations_list:
      img_id = str(annotation['id'])
      if img_id in imgs_dict:
        img_fp = os.path.join(img_folder, imgs_dict[img_id])
        imwidth,imheight = PIL.Image.open(img_fp).size
        if img_id not in data_dict:
          data_dict[img_id] = []

    self.x, self.ids = [], []
    for img_id in data_dict.keys():
      self.x.append( os.path.join(img_folder, imgs_dict[img_id]) )
      self.ids.append( img_id )

  def __len__(self):
    return int(np.ceil(len(self.x) / float(self.batch_size)))
  
  def __getitem__(self, idx):
    batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]

    x_acc = []
    original_img_dims = []
    with Pool(self.batch_size) as p:
      # Read in the PIL objects from filepaths
      batch_x = p.map(load_img, batch_x)
    
    for x in batch_x:
      W,H = x.size
      original_img_dims.append( (W,H) )

      x_aug = x
      x_acc.append( np.array(x_aug) )

    return self.get_batch_test(idx, x_acc, original_img_dims) if self.testmode else self.get_batch(x_acc, y_acc)

  def get_batch_test(self, idx, x_acc, original_img_dims):
    batch_ids = self.ids[idx * self.batch_size:(idx + 1) * self.batch_size]
    return batch_ids, original_img_dims, np.array( x_acc ) , 1

  def get_batch(self, x_acc, y_acc):
    return np.array( x_acc ), { dimkey: np.array( gt_tensor ) for dimkey, gt_tensor in y_acc.items() }

# Custom function to replace decode_tensor()
def unpack_preds(pred_dict):
  results = []
  det_num = pred_dict['num_detections']
  det_score = pred_dict['detection_scores']
  category_id = pred_dict['detection_classes']
  for i in range(det_num):
    predx, predy, predw, predh = pred_dict['detection_boxes'][i]
    results.append((det_score[i], category_id[i], predx, predy, predw, predh))
  return results

def aug_identity(pil_img, label_arr):
  return np.array(pil_img), label_arr

# aspect_ratios = [(1,1), (1,2), (1,3), (1,4)]  # model says aspect_ratios: [0.5, 1.0, 2.0]
input_shape = (600,800,3)

print("Loading eval images from:     ", eval_imgs_folder)
print("Loading eval annotations from:", eval_annotations)
# test_sequence = TILSequence(eval_imgs_folder, eval_annotations, 1, aug_identity)

Loading eval images from:      /content/drive/My Drive/Colab Notebooks/TIL/CV/input/val/val
Loading eval annotations from: /content/drive/My Drive/Colab Notebooks/TIL/CV/input/val.json


In [ ]:
# Copied over from Master code

# Generating detections on the folder of validation images
detections = []
det_threshold=0.
for i in tqdm(range(len(test_sequence))):
  ids_seq, dims, input_arr, _ = test_sequence[i]
  input_arr = input_arr[0]
  img_id = int(ids_seq[0])
  W,H = dims[0]

  # Here, I'm inferencing one-by-one, but you can batch it if you want it faster
  pred_dict = run_inference_for_single_image(input_arr, detection_graph)
  preds = unpack_preds(pred_dict)

  # Post-processing
  preds = [pred for pred in preds if pred[0] >= det_threshold]
  preds.sort( key=lambda x:x[0], reverse=True )
  preds = preds[:100] # we only evaluate you on 100 detections per image

  for i, pred in enumerate(preds):
    conf,cat_id,x,y,w,h = pred
    left = W * (x - w/2.)
    left = round(left,1)
    top = H * (y - h/2.)
    top = round(top,1)
    width = W*w
    width = round(width,1)
    height = H*h
    height = round(height,1)
    conf = float(conf)
    cat_id = int(cat_id)
    detections.append( {'image_id':img_id, 'category_id':cat_id, 'bbox':[left, top, width, height], 'score':conf} )


100%|██████████| 1474/1474 [1:36:08<00:00,  3.91s/it]
